In [58]:
import gensim.models as g
import codecs
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [44]:
#parameters
model = "Data/stage 3/pretrained_doc2vec/enwiki_dbow/doc2vec.bin"
expert_data = "Data/stage 3/cleaned-ukiyo-e-content-remove-nan-keyword-filtered.csv"
general_people_data = 'Data/stage 3/best_description_for_each_image.csv'

expert_data_with_doc2vec = 'Data/stage 3/expert_data_with_doc2vec.pkl'
general_ppl_data_with_doc2vec = 'Data/stage 3/general_ppl_data_with_doc2vec.pkl'

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

In [10]:
#load model
m = g.Doc2Vec.load(model)

In [45]:
def doc2vec(d):
    words = d.strip().split()
    vector = np.array([])
    for x in m.infer_vector(words, alpha=start_alpha, steps=infer_epoch):
        vector = np.append(vector, x)
    return vector

In [46]:
expert_df = pd.read_csv(expert_data)
expert_df['vector'] = expert_df.Description.apply(lambda d: doc2vec(d))
expert_df.to_pickle(expert_data_with_doc2vec)

In [47]:
general_ppl_df = pd.read_csv(general_people_data)
# general_ppl_df.head()
general_ppl_df['vector'] = general_ppl_df.description.apply(lambda d: doc2vec(d))
general_ppl_df.to_pickle(general_ppl_data_with_doc2vec)
general_ppl_df.head()

,image,description,vector
0,https://research.britishmuseum.org/collectioni...,The picture show a beautiful view of a very hi...,"[-0.09277264028787613, -0.0010732650989666581,..."
1,https://research.britishmuseum.org/collectioni...,"There are three person in the picture, two wom...","[-0.029725534841418266, -0.36949780583381653, ..."
2,https://research.britishmuseum.org/collectioni...,two woman are in the pictures. one of them is ...,"[-0.45402824878692627, -0.2185823619365692, -0..."
3,https://research.britishmuseum.org/collectioni...,There are two man fighting each other in the p...,"[-0.4591072201728821, -0.2511102855205536, -0...."
4,https://research.britishmuseum.org/collectioni...,A japanese woman is holding the kid. The woman...,"[-0.3618186414241791, -0.116968534886837, -0.3..."


In [97]:
expert_df = pd.read_pickle(expert_data_with_doc2vec)
general_ppl_df = pd.read_csv(general_people_data)
for i, row in general_ppl_df.iterrows():
    print(i, row.description, row.image)
    image = expert_df[
        expert_df.Source == row.image]

    vector = np.array([image.vector.values[0]])
    matrix = np.vstack(expert_df.vector.values)
    distances = cdist(vector, matrix)
    min = distances.argsort()

    for i in range(1, 6):
        print(expert_df.iloc[min[0]].Source.values[i])

0 The picture show a beautiful view of a very high mountain along with a wide river. The context looks like, there is mist covering everything, there is one boat in the river. https://research.britishmuseum.org/collectionimages/AN00759/AN00759298_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00759/AN00759304_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00759/AN00759308_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00759/AN00759280_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00759/AN00759294_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00231/AN00231464_001_l.jpg
1 There are three person in the picture, two womans and one kid. They appear in the the mountain context, one one woman is carrying her kid, and the other woman is doing laundry in the river. From a distance, there is a house appear in the right corner of the picture. https://research.britishmuseum.org/collectionimages/AN00530/AN0053098

https://research.britishmuseum.org/collectionimages/AN00281/AN00281353_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00530/AN00530207_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00605/AN00605390_001_l.jpg
16 The picture contains one man who is lifting a woman`s mask. The man is wearing no trousers, but with two swords. I think the season is on autumn, because there are many momiji on the ground. The man seems very sad, maybe it is because he just lost someone that he loves very much. https://research.britishmuseum.org/collectionimages/AN00588/AN00588993_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00588/AN00588995_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00589/AN00589129_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00589/AN00589109_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00589/AN00589119_001_l.jpg
https://research.britishmuseum.org/collectionimages/AN00589/AN005890